<a href="https://colab.research.google.com/github/whbpt/MetalNet/blob/master/Dataset_Metalloprotein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Dataset_Metalloprotein
Integrating all the informations contains protein coevolution information from bakerlab, and metal binding sites from PDB together.

In [0]:
##load google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive

In [0]:
import os
# requests for fetching html of website
import requests
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import tensorflow as tf
from scipy.spatial.distance import pdist,squareform
import pickle
from graphviz import Graph


## Get PDB dataset from [GREMLIN website](https://gremlin2.bakerlab.org/exceptions.php)

it is recycled data from [<Origins of coevolution between residues distant in protein 3D structures>](https://www.pnas.org/content/early/2017/08/03/1702664114)

In [0]:
%%bash
FILE="MetalNet_PDB" # create a project folder
if [ ! -d "$FILE" ]; then 
  mkdir $FILE
fi

In [0]:
# check to the default folder
cd /gdrive/My\ Drive/MetalNet_PDB/

In [0]:
%%bash
# download the PDB_dataset from gremlin website

if [ ! -f dataset_ivan.zip ];then
  wget https://gremlin2.bakerlab.org/exceptions/dataset_ivan.zip
fi
if [ ! -d data ];then
  unzip -q dataset_ivan.zip
fi

- .a3m.gz = original alignment
- .aln = positions with > 50% gaps removed
- .mtx = L*L matrix (output from gremlin)
- .mtx_ref = mapping of aln/mtx length to full sequence length
- .pdb = actual 3D protein
- .dist = distances extracted for each residue pairs. Typically < 5 we would consider to be a contact, but some people are more generious set to ~6

## Metals

### downloading PDB files from website
extracts the coordinates of each metal present filters thru each residue -- only residues that are <2.5 A from the metal (for each coordinate) are considered for the selected residues, calculate the distance between it and the metal only residues that are < 2.5 A total are kept append the residues that are < 2.5 A away from a specific metal to a single list gather the multiple lists (one list for each metal) make dict where protein name => (list of metal names, lists of associated residues)


In [0]:
def get_path(PDB_name,appendix,mode=0,remove_slash=True):
  '''model == 0 : data file
     model == 1 : the generated pdb_data file, no pollution on data file
     in Bash mode, the slash should be kept, because the bash is sensitive to the
     space.
     in Python mode， the slash should be removed.
  '''
  if mode ==0:
    seq = f"/gdrive/My\ Drive/MetalNet_PDB/data/{PDB_name[1:3]}/{PDB_name}.{appendix}"
  else :
    seq = f"/gdrive/My\ Drive/MetalNet_PDB/pdb_data/{PDB_name[1:3]}/{PDB_name}.{appendix}"
  if remove_slash == True:
    seq = seq.replace("\\","")
  return seq

In [0]:
# this metal dict only extracts residues that are next to the metal in a biological unit
pdb_list = pd.read_csv("list",header=None,sep=" ")
PDB_names = np.array(pdb_list)[:,0]
count = 0

In [0]:
%%bash
FILE="pdb_data"
if [ ! -d "$FILE" ]; then 
  mkdir $FILE
fi

In [0]:
### 
last_count = count
for PDB_name in PDB_names[last_count:]:
  folder_path = get_path(PDB_name,"",1)[0:-1]
  if os.path.exists(folder_path) == False:
    os.system(f"mkdir {folder_path}")
  pdb_path = get_path(PDB_name,"pdb",1)
  if os.path.exists(pdb_path) == False:
    pdb_path = get_path(PDB_name,"pdb",1,remove_slash=False)  
    print("downloading")
    os.system(f"wget -nc -O {pdb_path} https://files.rcsb.org/download/{PDB_name[:-1]}.pdb")
  count += 1
  if count %100 ==0:
    print(count)

In [0]:
ls pdb_data/??/*.pdb|wc

## Collecting labels
  


In [0]:
def get_coord(line):
  '''x,y,z'''
  return np.array([line[26:38], line[38:46], line[46:54]],dtype=np.float)

metal_types = ['ZN', 'CA', 'MG', 'CU', 'FE', 'NI', 'MN', 'SF4', 'FES', 'C1O', 'C2O', 'CUA']
hetero_atom = ['O', 'N', 'S'] # acceptable binding atoms in the residues
main_chain = ['CA', 'O', 'N', 'C']

In [0]:
PDB_metal_dict = {} # input is protein name; output is [[metal names],[related residues]]
dist_cutoff = 3 # Angstrom
file_name = "PDB_metal_dict.pkl"
redo_flag = False
if os.path.exists(file_name)==False or redo_flag == True:
  for count,PDB_name in enumerate(PDB_names[:-1]): # last element is a weird symbol (not a protein name!)
    PDB_chain = PDB_name[-1]
    if count%100==0:
      print(count)

    metal_coord = []
    resi_nums = []
    current_metals = [] # metal residue numbers that have already been recorded
    HETATMs = os.popen(f"grep ^HETATM {get_path(PDB_name, 'pdb',1,remove_slash = False)}").readlines()
    if len(HETATMs) >0:
      for line in HETATMs:
        if line[20:22].strip()==PDB_chain:
          metal_type = line[16:20].strip()
          if metal_type in metal_types:
            metal_name = line[22:26].strip()+"_"+metal_type
            if metal_name in current_metals:
              metal_coord[-1].append(get_coord(line))
            else:
              metal_coord.append([get_coord(line)])
              current_metals.append(metal_name)
      #print(current_metals,metal_coord)        
      assert len(current_metals) == len(metal_coord)
      fake_PDB = open(get_path(PDB_name, 'pdb'), 'r')

      for i in range(len(current_metals)):
        residues = set()
        for j in range(len(metal_coord[i])):
          for line in fake_PDB.readlines():
            if line[:4] == 'ATOM' and (line[13:17].strip() not in main_chain):
              resi_num = int(line[22:26].strip())
              atom_coord = get_coord(line)
              diff = atom_coord - metal_coord[i][j]      
              if np.sqrt(np.sum(np.square(diff))) < dist_cutoff:
                residues.add(resi_num)

        resi_nums.append(residues)
      assert len(resi_nums) == len(current_metals)
      metal = []
      resi = []
      for i in range(len(current_metals)):
        if len(resi_nums[i]) >= 3:
          metal.append(current_metals[i])
          resi.append(resi_nums[i])
      if len(metal)>0 :
        PDB_metal_dict[PDB_name] = [metal, resi]    
        print(PDB_name,metal,resi)
      fake_PDB.close()
  fileObject = open(file_name,'wb') # write as binary
  pickle.dump(PDB_metal_dict,fileObject)
  fileObject.close()
else:
  PDB_metal_dict = pickle.load(open(file_name,'rb'))

In [0]:
len(PDB_metal_dict)

## MSA similarity filter
  remove similar MSA by hhsuite, to reduce overfitting problems in training set
 
### method
   For each msa you need to first create an HMM. Since we need to do HMM-HMM alignment, for this we can use the HHsuite. I would recommend using version 2 (i had some issues with version 3... I suspect it might have some bugs).

You can download a pre-compiled version here:
http://wwwuser.gwdg.de/~compbiol/data/hhsuite/releases/all/hhsuite-2.0.16-linux-x86_64.tar.gz

warning: Since there are two software that make hmm(s) for sequence search (hmmer and hhsuite), and the formats are different. When making hmm(s) for hmmer we use extension of *.hmm, for hhsuite we use *.hhm. 

To make hmm(s) for hhsuite
hhmake -i ID.fas -o ID.hhm

then do (to concatenate all hhms into a single file)
cat ../*.hhm >  ALL.hhm

hhsearch -cpu 20 -v 0 -alt 1 -i ID.hhm -d ALL.hhm -o /dev/null -scores ID.sco

I typically copy the ALL.hhm to /dev/shm, which tends to speed up the search (since it's being read from memory).

### cutoff recommendation

- LOG-EVAL< 0  means totally random
- LOG-EVAL > 3  means same fold (Pfam)
- LOG-EVAL > 20  means same function
- LOG-EVAL > 80  means same [orthology](https://en.wikipedia.org/wiki/Sequence_homology#Orthology) (two copies in the same species)

In [0]:
# check to the default folder
cd /gdrive/My\ Drive/MetalNet_PDB/

In [0]:
##download the hhsuite
%%bash
FILE="hhsuite-2.0.16-linux-x86_64/"
if [ ! -d "$FILE" ]; then 
  wget -q -nc http://wwwuser.gwdg.de/~compbiol/data/hhsuite/releases/all/hhsuite-2.0.16-linux-x86_64.tar.gz
  tar -xzf hhsuite-2.0.16-linux-x86_64.tar.gz
  rm hhsuite-2.0.16-linux-x86_64.tar.gz
  chmod +x /gdrive/My\ Drive/MetalNet_PDB/hhsuite-2.0.16-linux-x86_64/bin/*
  fi

In [0]:
fileObject = open("metal_list",'w')
fileObject.write("\n".join(list(PDB_metal_dict.keys())))
fileObject.close()

In [0]:
%%bash
export HHLIB=/gdrive/My\ Drive/MetalNet_PDB/hhsuite-2.0.16-linux-x86_64
for gene in `cat metal_list`;do
  export folder=`echo $gene|cut -c 2-3`
  export input1="data/$folder/$gene.a3m.gz"
  export output1="pdb_data/$folder/$gene.a3m"
  export output2="pdb_data/$folder/$gene.hhm"
  if [ ! -f $output2 ];then
    gunzip -c $input1 > $output1
    /gdrive/My\ Drive/MetalNet_PDB/hhsuite-2.0.16-linux-x86_64/bin/hhmake -i $output1 -o $output2
  fi
done

In [0]:
!ls pdb_data/??/*.hhm|wc

In [0]:
%%bash
if [ ! -f "pdb_data/all.hhm" ];then
  cat pdb_data/??/*.hhm >>pdb_data/all.hhm
fi

In [0]:
%%bash
export HHLIB=/gdrive/My\ Drive/MetalNet_PDB/hhsuite-2.0.16-linux-x86_64
for gene in `cat metal_list`;do
  export folder=`echo $gene|cut -c 2-3`
  export input="pdb_data/$folder/$gene.hhm"
  export output="pdb_data/$folder/$gene.score"
  if [ ! -f $output ];then
    /gdrive/My\ Drive/MetalNet_PDB/hhsuite-2.0.16-linux-x86_64/bin/hhsearch -cpu 20 -v 0 -alt 1 -i $input -d pdb_data/all.hhm -o /dev/null -scores $output
  fi
done

In [0]:
!ls pdb_data/??/*.score|wc

In [0]:
uniq_pkl = "uniq.pkl"
if os.path.exists(uniq_pkl) == False: 
  uniq_set = set()
  for PDB_name in PDB_metal_dict.keys():
    uniq_set.add(PDB_name)
  count = 0
  for PDB_name in PDB_metal_dict.keys():
    count = count + 1
    if count %20==0:
      print(count)
    try:
      
      repeat_flag=0
      with open(get_path(PDB_name,"score",1),'r') as f:
        similarity = pd.read_csv(f,header=4,sep=r"\s*")
        log_eval = np.array(similarity)[:,-2].astype(np.float)
        idx = np.where(log_eval> 20)[0]
        if len(idx)>1:
          idx = idx[1:]
          gene_list =  np.array(similarity)[:,0][idx]
          for gene in gene_list:
            if (gene in uniq_set) and (repeat_flag)==0:
              repeat_flag=1
            if repeat_flag==1:
              try:
                uniq_set.remove(gene)
              except:
                continue
    except OSError:
      raise
    except:
      raise
  pickle.dump(uniq_set,open(uniq_pkl,'wb'))
else:
  uniq_set = pickle.load(open(uniq_pkl,'rb'))

In [0]:
len(uniq_set)

##Collecting all the information related with metal binding proteins

In [0]:
##creat a PROTEIN class to store metal binding sites and sequence 
class PROTEIN:
  def __init__(self):
    self.metal_dict = {}
    self.sequence = str()
    self.gene_name = str()
    self.gap_dict = {}
    self.CHED_seq = str()
    self.CHED_dict = {}
    self.metal_mask = []
    self.metal_seq = []
    self.GREMLIN_CM = []

In [0]:
Tome_pkl = "Tome.pkl"
redo_flag = True
if os.path.exists(Tome_pkl) == False or redo_flag == True: 
  CHED_list = ["C","H","E","D"]
  metal_sum = 0
  Tome_dict = {}
  for pdb_count,PDB_ID in enumerate(uniq_set):
    if pdb_count%10==0:
      print(pdb_count)
    Tome_dict.setdefault(PDB_ID,PROTEIN())
    Protein = Tome_dict[PDB_ID]
    Protein.gene_name = PDB_ID
    # the gap has removed in MSA file when fed into GREMLIN,so needs do some alignment
    ref_contents = open(get_path(PDB_ID,"mtx_ref"),'r').readlines()
    pdb_seq = ref_contents[0].strip()
    GREMLIN_seq = ref_contents[1].strip()
    Protein.sequence = pdb_seq 

    [metals,sites] = PDB_metal_dict[PDB_ID]

    ncol = len(pdb_seq)

    # load the CHED mask
    CHED_mask = np.zeros(ncol)
    for i,AA in enumerate(pdb_seq):
      if AA in CHED_list:
        CHED_mask[i]=1

    CHED_idx = np.where(CHED_mask>0)[0]
    Protein.CHED_seq = "".join(([pdb_seq[AA] for AA in CHED_idx]))
    CHED_dict = {key:i for (i,key) in enumerate(CHED_idx)}
    Protein.CHED_dict = CHED_dict

    CHED_ncol = len(CHED_idx)##a small matrix only with CHED
    metal_mask = np.zeros((CHED_ncol,CHED_ncol))

    for i,metal in enumerate(metals):
      Protein.metal_dict.setdefault(metal,[])
      metal_seq = np.zeros((CHED_ncol))
      for site in sites[i]:
        AA = pdb_seq[int(site)]
        if AA in CHED_list:
          site_tag = f"{site}_{AA}"
          Protein.metal_dict[metal].append(site_tag)
          metal_seq[CHED_dict[site]] = 1
      metal_mask += metal_seq[None,:] * metal_seq[:,None]
    
    np.fill_diagonal(metal_mask,0)
    Protein.metal_mask = metal_mask
    Protein.metal_seq = metal_seq

    ### generate gap dictionary to map the alignment
    AA_count = 0
    gap_dict=dict()
    for i,AA in enumerate(GREMLIN_seq):
      if AA!="-":
        gap_dict[AA_count] = i
        AA_count+=1
    Protein.gap_dict = gap_dict
    
    # load the predicted GREMLIN Contact Map    
    GREMLIN_CM = np.zeros((CHED_ncol,CHED_ncol))
    mtx_file = get_path(PDB_ID,"mtx",0)
    GREMLIN_result = np.array(pd.read_csv(mtx_file,sep='\t',header=None))
    GREMLIN_result= GREMLIN_result[:,0:-1]
    temp = GREMLIN_result.flatten()
    cutoff = sorted(temp,reverse=True)[4*ncol]  #top 2L 
    GREMLIN_result = GREMLIN_result > cutoff

    for i in range(GREMLIN_result.shape[0]):
      ori_i = gap_dict[i]
      if ori_i in CHED_idx:
        for j in range(GREMLIN_result.shape[0]):
          ori_j = gap_dict[j]
          if ori_j in CHED_idx:
            GREMLIN_CM[CHED_dict[ori_i],
                       CHED_dict[ori_j]] =  GREMLIN_result[i,j]
            
    np.fill_diagonal(GREMLIN_CM,0)
    Protein.GREMLIN_CM = GREMLIN_CM



  # remove those 
  temp = list(Tome_dict.keys())
  for gene in temp:
    metal_dict = Tome_dict[gene].metal_dict
    temp2 = list(metal_dict.keys())
    for metal in temp2:
      if len(metal_dict[metal])<3:
        metal_dict.pop(metal)
    if len(metal_dict)==0:
      Tome_dict.pop(gene)
      
  pickle.dump(Tome_dict,open(Tome_pkl,'wb'))
else:
  Tome_dict = pickle.load(open(Tome_pkl,'rb'))